In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


### Import the QmioBackend code

In [2]:
from qmiotools.integrations.qiskitqmio import QmioBackend

In [5]:
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit.visualization import plot_gate_map, plot_error_map

### Create an instance of the QMIO backend, loading he last calibration from the directory given by the QMIO_CALIBRATIONS environ

In [4]:
backend=QmioBackend("/home/cesga/agomez/QMIO/qmio/qmiotools/calibrations/2024_05_11__13_27_21.json")

Importing calibrations from  /home/cesga/agomez/QMIO/qmio/qmiotools/calibrations/2024_05_11__13_27_21.json


KeyError: 'T1 (s)'

In [ ]:
%matplotlib inline
c=QuantumCircuit(2)
c.h(0)
c.cx(0,1)
c.measure_all()
c.draw(output="mpl")

In [ ]:
d=c.copy()
d.draw(output="mpl")

In [5]:

qc_basis = transpile(c, backend)
qc_basis.draw(output="mpl")

NameError: name 'c' is not defined

In [6]:
qd_basis = transpile(d, backend)
qd_basis.draw(output="mpl")

NameError: name 'd' is not defined

# These cells only work if executed in a node with connection to the QPU

In [ ]:
job=backend.run([c,d], shots=10000)
job.result().get_counts(c)
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Without Optimisation")

In [ ]:
job=backend.run([qc_basis,qd_basis])
print(job.result().get_counts(qc_basis))
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(),title="With Optimisation")

In [ ]:
print(backend.target.build_coupling_map())

In [ ]:
import numpy as np
qc = QuantumCircuit(2, 2)
 
qc.h(0)
qc.x(1)
qc.cp(np.pi/4, 0, 1)
qc.h(0)
qc.measure([0], [0])
qc.measure([1], [1])
 
qc_basis = transpile(qc, backend)
qc_basis.name=c.name+"_opt"
qc_basis.draw(output="mpl")

In [ ]:
job=backend.run([qc,qc_basis])
job.result().get_counts(qc)
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Comparing Optimisation")

In [ ]:
nqubits=3
ghz = QuantumCircuit(nqubits)
ghz.h(0)
ghz.cx(0, range(1, nqubits))
ghz.measure_all()
try:
    ghz_0=transpile(ghz,backend,optimization_level=0)
    job=backend.run(ghz_0)
    job.result().get_counts(ghz_0)
    plot_histogram(job.result().get_counts(), title="Comparing Optimisation")
except:
    print("ERROR: No está arreglado este punto")

In [ ]:
ghz_1=transpile(ghz,backend,optimization_level=1)
ghz_1.name=ghz_1.name+"_1"
ghz_2=transpile(ghz,backend,optimization_level=2)
ghz_2.name=ghz_1.name+"_2"
ghz_3=transpile(ghz,backend,optimization_level=3)
ghz_3.name=ghz_1.name+"_3"
job=backend.run([ghz_0,ghz_1,ghz_2,ghz_3])
plot_histogram(job.result().get_counts(), title="Comparing Optimisation",legend=[ghz_0.name,ghz_1.name,ghz_2.name,ghz_3.name])

In [4]:
from qmiotools.data import QUBIT_POSITIONS
pos=QUBIT_POSITIONS
backend=QmioBackend()
def plot_t1_colors(backend):
    import matplotlib as mpl
    from matplotlib.cm import ScalarMappable as SC
    from matplotlib import colors

    cmap = mpl.colormaps['plasma']

    sc=SC(cmap=cmap)

    T1s=[backend.qubit_properties(i).t1*1e4 for i in range(32)]
    
    col=sc.to_rgba(T1s)
    q=[colors.to_hex(i) for i in col]
    return q

    

plot_gate_map(backend, qubit_coordinates=pos,plot_directed=True,line_width=1,qubit_color=plot_t1_colors(backend))

Importing calibrations from  /opt/cesga/qmio/hpc/calibrations/2024_11_15__12_00_02.json


CalledProcessError: Command '['neato', '-T', 'png']' returned non-zero exit status 1.

In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap
 
#coupling_map = CouplingMap.from_heavy_hex(3)

 
ghz = QuantumCircuit(15)
ghz.h(0)
ghz.cx(0, range(1, 15))
 
depths = []
for i in range(20):
    depths.append(
        transpile(
            ghz,
            backend,
            seed_transpiler=i,
            optimization_level=1,
            #layout_method='trivial'  # Fixed layout mapped in circuit order
        ).depth()
    )
 
plt.figure(figsize=(8, 6))
plt.hist(depths, align='left', color='#AC557C')
plt.xlabel('Depth', fontsize=14)
plt.ylabel('Counts', fontsize=14);

In [ ]:
n=3
wh = QuantumCircuit(n)
for i in range(n):
    wh.h(i)
wh.measure_all()
wh=transpile(wh,backend, optimization_level=2)
#wh.draw(output="mpl")

In [ ]:
job=backend.run(wh,shots=20000,repetition_period=400e-6)
plot_histogram(job.result().get_counts())

In [ ]:
job=backend.run(wh,shots=20000,repetition_period=200e-6)
plot_histogram(job.result().get_counts())

In [6]:
plot_gate_map(backend, qubit_coordinates=pos,line_width=1,qubit_color=plot_t1_colors(backend))

CalledProcessError: Command '['neato', '-T', 'png']' returned non-zero exit status 1.

In [7]:
from qiskit.visualization import plot_coupling_map
 
num_qubits = 8
qubit_coordinates = [[0, 1], [1, 1], [1, 0], [1, 2], [2, 0], [2, 2], [2, 1], [3, 1]]
coupling_map = [[0, 1], [1, 2], [2, 3], [3, 5], [4, 5], [5, 6], [2, 4], [6, 7]]
plot_coupling_map(num_qubits, qubit_coordinates, coupling_map)

CalledProcessError: Command '['neato', '-T', 'png']' returned non-zero exit status 1.